In [180]:
from pydantic import BaseModel
from typing import List, Dict, Tuple
import sys
from enum import Enum
import requests
import random

In [181]:
# Scoring Radii

class DartboardRadius(Enum):
    """Enum representing the radius thresholds for different dartboard scoring areas."""
    INNER_BULL = 0.025
    OUTER_BULL = 0.065
    INNER_TRIPLE = 0.40
    OUTER_TRIPLE = 0.43
    INNER_DOUBLE = 0.65
    OUTER_DOUBLE = 0.68

# Scores / Scoring Multipliers
class ScoreMultipliers(Enum):
    SINGLE = 1
    DOUBLE = 2
    TRIPLE = 3
    INNER = 50
    OUTER = 25

# Dartboard segment definitions
SEGMENTS = [6, 13, 4, 18, 1, 20, 5, 12, 9, 14, 11, 8, 16, 7, 19, 3, 17, 2, 15, 10]
SEGMENT_ANGLE = 360 / len(SEGMENTS)  # 18 degrees per segment
ANGLE_OFFSET = 9  # Align 0° to the midpoint of segment 6


In [168]:
class Throw(BaseModel):
    target: int | None = None
    radius: float
    angle: float

    @classmethod
    def from_score(cls, zone: int, multiplier: int):
        radius = 0.5
        return cls(radius=radius, angle=angle)

class ThrowTriplet(BaseModel):
    throws: list[Throw]

class ThrowScore(BaseModel):
    multiplier: float
    zone: int
    score: int

In [ ]:
class Throw(BaseModel):
    target: int | None = None
    radius: float
    angle: float

    @classmethod
    def from_score(cls, zone: int, multiplier: ScoreMultiplier):
        if (multiplier == multiplier.INNER_BULL):
            zone = int(random.uniform(1, 20))
            radius = random.uniform(0, DartboardRadius.INNER_BULL.value)
        

        if multiplier == multiplier.OUTER_BULLSEYE:
            zone = int(random.uniform(1, 20))
            radius = random.uniform(0, DartboardRadius.INNER_BULL.value)
        angle = inverse_map_segment_to_angle(zone)

        return cls(radius=radius, angle=angle)

In [187]:
random.uniform(0, DartboardRadius.INNER_BULL.value)

0.0020706040646339685

In [173]:

def map_angle_to_segment(angle: float) -> int:
    """
    Maps a given angle to a dartboard segment.

    Parameters
    ----------
    angle : float
        The angle of the dart throw in degrees.

    Returns
    -------
    int
        The corresponding dartboard segment number.
    """
    adjusted_angle = (angle + ANGLE_OFFSET) % 360
    segment_index = int(adjusted_angle // SEGMENT_ANGLE) % len(SEGMENTS)
    return SEGMENTS[segment_index]

def inverse_map_segment_to_angle(zone: int) -> float:
    """
    Given a dartboard segment (zone), return a random angle that would fall within that segment.

    Parameters
    ----------
    zone : int
        The target dartboard segment (1-20).

    Returns
    -------
    float
        A random angle within the valid range for the given segment.
    """
    if zone not in SEGMENTS:
        raise ValueError(f"Invalid dartboard zone: {zone}")

    segment_index = SEGMENTS.index(zone)
    min_angle = segment_index * SEGMENT_ANGLE - ANGLE_OFFSET
    max_angle = (segment_index + 1) * SEGMENT_ANGLE - ANGLE_OFFSET

    # Ensure angles wrap correctly around 360 degrees
    min_angle %= 360
    max_angle %= 360

    # Generate a random angle within the valid segment range
    if min_angle > max_angle:  # Handle wrap-around case (e.g., segment near 0°)
        angle = random.uniform(min_angle, max_angle + 360) % 360
    else:
        angle = random.uniform(min_angle, max_angle)

    return angle


In [178]:
random.uniform(0.5,0.6)

0.513297312670815

In [82]:

def calculate_score(throw: Throw) -> ThrowScore:
    """
    Computes the score of a dart throw based on its radius and angle.

    Parameters
    ----------
    throw : Throw
        The dart throw object containing radius and angle.

    Returns
    -------
    ThrowScore
        The calculated score, multiplier, and zone.
    """
    angle = throw.angle % 360  # Normalize angle
    radius = throw.radius

    if radius <= INNER_BULL_RADIUS:
        return ThrowScore(multiplier=SINGLE, zone="Inner Bullseye", score=50)

    if radius <= OUTER_BULL_RADIUS:
        return ThrowScore(multiplier=SINGLE, zone="Outer Bullseye", score=25)

    segment = map_angle_to_segment(angle)

    if INNER_TRIPLE_RADIUS <= radius <= OUTER_TRIPLE_RADIUS:
        return ThrowScore(multiplier=TRIPLE, zone=segment, score=segment * TRIPLE)

    if INNER_DOUBLE_RADIUS <= radius <= OUTER_DOUBLE_RADIUS:
        return ThrowScore(multiplier=DOUBLE, zone=segment, score=segment * DOUBLE)

    if radius > OUTER_DOUBLE_RADIUS:
        return ThrowScore(multiplier=SINGLE, zone="Miss", score=0)

    return ThrowScore(multiplier=SINGLE, zone=segment, score=segment * SINGLE)


In [159]:
class AroundTheWorld:
    def __init__(self, mode: str, darts_per_target: int):
        """
        Initialize the game with the scoring mode and darts per target.

        Parameters
        ----------
        mode : str
            The scoring mode ("singles", "doubles", "triples").
        darts_per_target : int
            Number of darts required per target (1 or 3).
        """
        # TODO - change the mode to an ENUM in constants
        if mode not in {"singles", "doubles", "triples"}:
            raise ValueError("Mode must be one of 'singles', 'doubles', or 'triples'.")
        if darts_per_target not in {1, 2, 3}:
            raise ValueError("Darts per target must be 1 or 3.")

        self.mode = mode
        self.darts_per_target = darts_per_target
        self.current_target = 1
        self.throws_record: list[ThrowTriplet] = []  # Records all ThrowTriplets
        self.completed_targets: list[int] = []  # List of completed targets



    def _check_target(self, throw):
        return
    
    def process_throw(self, triplet: ThrowTriplet):
        self.throws_record.append(triplet)
        scores = [calculate_score(throw) for throw in triplet.throws]
        if self.darts_per_target == 1:
            for score in scores:
                if score.zone == self.current_target:
                    self.completed_targets.append(self.current_target)
                    self.current_target += 1
        else: 
            if zones.count(self.current_target) == self.darts_per_target:
                self.completed_targets.append(self.current_target)
                self.current_target += 1

        self.throws_record.append(triplet)
        if self.current_target > 20:
            print("YOU WIN - GOOD JOB")


        

    # def _get_required_multiplier(self) -> int:
    #     """
    #     Get the multiplier required for the current mode.

    #     Returns
    #     -------
    #     int
    #         The multiplier (1, 2, or 3) corresponding to singles, doubles, or triples.
    #     """
    #     return {"singles": 1, "doubles": 2, "triples": 3}[self.mode]

    # def _is_target_achieved(self, throw_triplet: ThrowTriplet) -> bool:
    #     """
    #     Check if the target in the given ThrowTriplet is achieved.

    #     Parameters
    #     ----------
    #     throw_triplet : ThrowTriplet
    #         The set of throws to evaluate.

    #     Returns
    #     -------
    #     bool
    #         True if the target is achieved, False otherwise.
    #     """
    #     required_target = int(throw_triplet.target)
    #     hits = sum(
    #         1 for throw in throw_triplet.throws if self._is_hit(throw, required_target)
    #     )
    #     return hits >= self.darts_per_target

    # def _is_hit(self, throw: Throw, target: int) -> bool:
    #     """
    #     Determine if a single throw hits the target with the required multiplier.

    #     Parameters
    #     ----------
    #     throw : Throw
    #         The dart throw to evaluate.
    #     target : int
    #         The target number.

    #     Returns
    #     -------
    #     bool
    #         True if the throw is a hit, False otherwise.
    #     """
    #     # Placeholder logic: Replace with actual logic based on radius and angle
    #     # This assumes a simplified function where angle determines the segment
    #     segment = round((throw.angle + 360) % 360 / 18) + 1
    #     return segment == target

    # def process_triplet(self, throw_triplet: ThrowTriplet) -> None:
    #     """
    #     Process a set of three throws and update the game state.

    #     Parameters
    #     ----------
    #     throw_triplet : ThrowTriplet
    #         The triplet of throws to process.
    #     """
    #     if int(throw_triplet.target) != self.current_target:
    #         raise ValueError(f"Incorrect target. Current target is {self.current_target}.")

    #     self.throws_record.append(throw_triplet)

    #     if self._is_target_achieved(throw_triplet):
    #         self.completed_targets.append(self.current_target)
    #         print(f"Target {self.current_target} achieved!")
    #         self.current_target += 1  # Advance to the next target

    # def is_game_over(self) -> bool:
    #     """
    #     Check if the game is over.

    #     Returns
    #     -------
    #     bool
    #         True if all targets are completed, False otherwise.
    #     """
    #     return self.current_target > 20

    # def get_game_status(self) -> Dict:
    #     """
    #     Get the current status of the game.

    #     Returns
    #     -------
    #     dict
    #         A dictionary containing game status.
    #     """
    #     return {
    #         "current_target": self.current_target,
    #         "completed_targets": self.completed_targets,
    #         "throws_record": self.throws_record,
    #         "mode": self.mode,
    #         "darts_per_target": self.darts_per_target,
    #     }


In [165]:
triplet = ThrowTriplet( 
    throws=[
        Throw(radius=0.3, angle=79),  # First throw
        Throw(radius=0.5, angle=50.0),  # Second throw
        Throw(radius=0.2, angle=135.0)  # Third throw
    ]
)

In [166]:
t = Throw(radius=0.3, angle=79)

In [167]:
vars(t)

{'target': None, 'radius': 0.3, 'angle': 79.0}

{'radius': 0.4, 'angle': 79.0}

In [154]:
game = AroundTheWorld(mode='singles', darts_per_target=2)

In [156]:
game.process_throw(triplet)
print(game.current_target)

1


In [142]:
zones = [calculate_score(throw).zone for throw in triplet.throws]

In [143]:
zones

[1, 18, 9]

In [109]:
current = 1

for zone in zones:
    print(f"current target: {current}")
    if zone == current:
        current += 1



current target: 1
current target: 1
current target: 2


In [2]:
import requests

In [10]:
# API endpoint URL
url = "http://127.0.0.1:8000/scores/throw"

# Send a GET request to the endpoint
response = requests.get(url, params={'radius':0.6, 'angle':90})

# Check the response
if response.status_code == 200:
    # Success: Print the response JSON
    print("Response JSON:", response.json())
else:
    # Error: Print the status code and error message
    print("Error:", response.status_code, response.text)

Response JSON: {'multiplier': 3, 'zone': 20, 'score': 60}
